In [1]:
# Importing the required libraries and dependencies
import pandas as pd

In [47]:
# *FOR GOOGLE COLAB* - Import the `files` library to allow files upload
#from google.colab import files

Mounted at /content/drive


In [5]:
# *FOR GOOGLE COLAB* - Upload "consolidated_seven_ny_counties.csv" into Colab, then store in a DataFrame
#uploaded = files.upload()

Saving consolidated_seven_ny_counties.csv to consolidated_seven_ny_counties.csv


In [2]:
# Storing the data in a Pandas DataFrame
# Setting the "date_local" column as the Datetime Index.

new_df = pd.read_csv(
    "Resources/consolidated_seven_ny_counties.csv",
    index_col='date_local',
    low_memory=False,
    parse_dates=True
)

# Preview the dataset
new_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Resources/consolidated_seven_ny_counties.csv'

In [3]:
new_df.shape

NameError: name 'new_df' is not defined

In [6]:
#display options to show all rows
pd.set_option('display.max_rows', None)

In [7]:
new_df.columns

Index(['county_code', 'parameter_code', 'parameter', 'latitude', 'longitude',
       'sample_duration_code', 'units_of_measure', 'observation_count',
       'validity_indicator', 'arithmetic_mean',
       ...
       'units_of_measure_81102', 'observation_count_81102',
       'validity_indicator_81102', 'arithmetic_mean_81102',
       'first_max_value_81102', 'first_max_hour_81102', 'aqi_81102',
       'county_81102', 'city_81102', 'aqi_max'],
      dtype='object', length=106)

In [17]:
#Dropping all unnecessary columns and displaying DataFrame
dropped_columns_df = new_df [['latitude_88502', 'longitude_88502','county_88502', 'aqi','aqi_42401','aqi_44201','aqi_42602','aqi_88502','aqi_max' ]].copy()
dropped_columns_df.rename(columns={"aqi": "aqi_42101", 'latitude_88502': 'latitude', 'longitude_88502': 'longitude', 'county_88502': 'county'}, inplace=True)
dropped_columns_df.head()

,latitude,longitude,county,aqi_42101,aqi_42401,aqi_44201,aqi_42602,aqi_88502,aqi_max
date_local,,,,,,,,,
2013-01-01,40.81551,-73.88553,Bronx,4.5,9.8,22.000,23.0,52.0,52.0
2013-01-02,40.81551,-73.88553,Bronx,6.5,11.4,21.000,26.5,60.0,60.0
2013-01-03,40.81551,-73.88553,Bronx,6.5,15.0,19.375,35.5,59.0,59.0
2013-01-04,40.81551,-73.88553,Bronx,7.0,13.0,19.000,36.5,60.0,64.5
2013-01-05,NaN,NaN,NaN,8.0,10.2,23.500,41.5,NaN,41.5


In [18]:
# View sum of null values per column
dropped_columns_df.isnull().sum()

latitude       692
longitude      692
county         692
aqi_42101     7180
aqi_42401    17756
aqi_44201    12579
aqi_42602    19335
aqi_88502      692
aqi_max          0
dtype: int64

In [19]:
#Fill in blanks in lat, long, and city columns using forward fill
dropped_columns_df['latitude'] = dropped_columns_df['latitude'].fillna(method='ffill')
dropped_columns_df['longitude'] = dropped_columns_df['longitude'].fillna(method='ffill')
dropped_columns_df['county'] = dropped_columns_df['county'].fillna(method='ffill')
dropped_columns_df.isnull().sum()

C:\Users\Chearine\AppData\Local\Temp\ipykernel_25228\2718495195.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dropped_columns_df['latitude'] = dropped_columns_df['latitude'].fillna(method='ffill')
C:\Users\Chearine\AppData\Local\Temp\ipykernel_25228\2718495195.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dropped_columns_df['longitude'] = dropped_columns_df['longitude'].fillna(method='ffill')
C:\Users\Chearine\AppData\Local\Temp\ipykernel_25228\2718495195.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dropped_columns_df['county'] = dropped_columns_df['county'].fillna(method='ffill')


latitude         0
longitude        0
county           0
aqi_42101     7180
aqi_42401    17756
aqi_44201    12579
aqi_42602    19335
aqi_88502      692
aqi_max          0
dtype: int64

In [20]:
#To fill the missing AQI_42101 score values in the provided code, we used imputation techniques.
#Based on the nature of air quality data, which often has temporal and spatial dependencies, we useds a combination of methods.

#1. Forward fill and backward fill:
# First, we’ll use forward fill and backward fill methods to handle missing values that occur in time series data.

# Forward fill and backward fill
dropped_columns_df['aqi_42101'] = dropped_columns_df['aqi_42101'].fillna(method='ffill').fillna(method='bfill')

#2. Interpolation:
#For any remaining missing values, we can use interpolation, which is particularly useful for time series data.
dropped_columns_df['aqi_42101'] = dropped_columns_df['aqi_42101'].interpolate(method='time')

#3. Mean imputation by category:
# If there are still missing values after the above steps, we can use mean imputation based on categories like ‘County’ and ‘date_local’.
# Group by relevant categories and fill with mean
dropped_columns_df['aqi_42101'] = dropped_columns_df.groupby(['county','date_local'])['aqi_42101'].transform(lambda x: x.fillna(x.mean()))

# 4. Overall mean imputation:
# As a last resort, fill any remaining missing values with the overall mean.
dropped_columns_df['aqi_42101'] = dropped_columns_df['aqi_42101'].fillna(dropped_columns_df['aqi_42101'].mean())

# # Check for missing values
print("Missing values before imputation:")
print(dropped_columns_df.isnull().sum())

C:\Users\Chearine\AppData\Local\Temp\ipykernel_25228\2345176665.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dropped_columns_df['aqi_42101'] = dropped_columns_df['aqi_42101'].fillna(method='ffill').fillna(method='bfill')


Missing values before imputation:
latitude         0
longitude        0
county           0
aqi_42101        0
aqi_42401    17756
aqi_44201    12579
aqi_42602    19335
aqi_88502      692
aqi_max          0
dtype: int64


In [21]:
#To fill the missing AQI_42401 score values in the provided code, we used imputation techniques.
#Based on the nature of air quality data, which often has temporal and spatial dependencies, we useds a combination of methods.

#1. Forward fill and backward fill:
# First, we’ll use forward fill and backward fill methods to handle missing values that occur in time series data.

# Forward fill and backward fill
dropped_columns_df['aqi_42401'] = dropped_columns_df['aqi_42401'].fillna(method='ffill').fillna(method='bfill')

#2. Interpolation:
#For any remaining missing values, we can use interpolation, which is particularly useful for time series data.
dropped_columns_df['aqi_42401'] = dropped_columns_df['aqi_42401'].interpolate(method='time')

#3. Mean imputation by category:
# If there are still missing values after the above steps, we can use mean imputation based on categories like ‘County’ and ‘date_local’.
# Group by relevant categories and fill with mean
dropped_columns_df['aqi_42401'] = dropped_columns_df.groupby(['county','date_local'])['aqi_42401'].transform(lambda x: x.fillna(x.mean()))

# 4. Overall mean imputation:
# As a last resort, fill any remaining missing values with the overall mean.
dropped_columns_df['aqi_42401'] = dropped_columns_df['aqi_42401'].fillna(dropped_columns_df['aqi_42401'].mean())

# # Check for missing values
print("Missing values before imputation:")
print(dropped_columns_df.isnull().sum())

C:\Users\Chearine\AppData\Local\Temp\ipykernel_25228\3272852146.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dropped_columns_df['aqi_42401'] = dropped_columns_df['aqi_42401'].fillna(method='ffill').fillna(method='bfill')


Missing values before imputation:
latitude         0
longitude        0
county           0
aqi_42101        0
aqi_42401        0
aqi_44201    12579
aqi_42602    19335
aqi_88502      692
aqi_max          0
dtype: int64


In [22]:
#To fill the missing AQI_44201 score values in the provided code, we used imputation techniques.
#Based on the nature of air quality data, which often has temporal and spatial dependencies, we useds a combination of methods.

#1. Forward fill and backward fill:
# First, we’ll use forward fill and backward fill methods to handle missing values that occur in time series data.

# Forward fill and backward fill
dropped_columns_df['aqi_44201'] = dropped_columns_df['aqi_44201'].fillna(method='ffill').fillna(method='bfill')

#2. Interpolation:
#For any remaining missing values, we can use interpolation, which is particularly useful for time series data.
dropped_columns_df['aqi_44201'] = dropped_columns_df['aqi_44201'].interpolate(method='time')

#3. Mean imputation by category:
# If there are still missing values after the above steps, we can use mean imputation based on categories like ‘County’ and ‘date_local’.
# Group by relevant categories and fill with mean
dropped_columns_df['aqi_44201'] = dropped_columns_df.groupby(['county','date_local'])['aqi_44201'].transform(lambda x: x.fillna(x.mean()))

# 4. Overall mean imputation:
# As a last resort, fill any remaining missing values with the overall mean.
dropped_columns_df['aqi_44201'] = dropped_columns_df['aqi_44201'].fillna(dropped_columns_df['aqi_44201'].mean())

# # Check for missing values
print("Missing values before imputation:")
print(dropped_columns_df.isnull().sum())

C:\Users\Chearine\AppData\Local\Temp\ipykernel_25228\28589287.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dropped_columns_df['aqi_44201'] = dropped_columns_df['aqi_44201'].fillna(method='ffill').fillna(method='bfill')


Missing values before imputation:
latitude         0
longitude        0
county           0
aqi_42101        0
aqi_42401        0
aqi_44201        0
aqi_42602    19335
aqi_88502      692
aqi_max          0
dtype: int64


In [23]:
#To fill the missing AQI_42602 score values in the provided code, we used imputation techniques.
#Based on the nature of air quality data, which often has temporal and spatial dependencies, we useds a combination of methods.

#1. Forward fill and backward fill:
# First, we’ll use forward fill and backward fill methods to handle missing values that occur in time series data.

# Forward fill and backward fill
dropped_columns_df['aqi_42602'] = dropped_columns_df['aqi_42602'].fillna(method='ffill').fillna(method='bfill')

#2. Interpolation:
#For any remaining missing values, we can use interpolation, which is particularly useful for time series data.
dropped_columns_df['aqi_42602'] = dropped_columns_df['aqi_42602'].interpolate(method='time')

#3. Mean imputation by category:
# If there are still missing values after the above steps, we can use mean imputation based on categories like ‘County’ and ‘date_local’.
# Group by relevant categories and fill with mean
dropped_columns_df['aqi_42602'] = dropped_columns_df.groupby(['county','date_local'])['aqi_42602'].transform(lambda x: x.fillna(x.mean()))

# 4. Overall mean imputation:
# As a last resort, fill any remaining missing values with the overall mean.
dropped_columns_df['aqi_42602'] = dropped_columns_df['aqi_42602'].fillna(dropped_columns_df['aqi_42602'].mean())

# # Check for missing values
print("Missing values before imputation:")
print(dropped_columns_df.isnull().sum())

C:\Users\Chearine\AppData\Local\Temp\ipykernel_25228\2642964399.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dropped_columns_df['aqi_42602'] = dropped_columns_df['aqi_42602'].fillna(method='ffill').fillna(method='bfill')


Missing values before imputation:
latitude       0
longitude      0
county         0
aqi_42101      0
aqi_42401      0
aqi_44201      0
aqi_42602      0
aqi_88502    692
aqi_max        0
dtype: int64


In [24]:
#To fill the missing AQI_88502 score values in the provided code, we used imputation techniques.
#Based on the nature of air quality data, which often has temporal and spatial dependencies, we useds a combination of methods.

#1. Forward fill and backward fill:
# First, we’ll use forward fill and backward fill methods to handle missing values that occur in time series data.

# Forward fill and backward fill
dropped_columns_df['aqi_88502'] = dropped_columns_df['aqi_88502'].fillna(method='ffill').fillna(method='bfill')

#2. Interpolation:
#For any remaining missing values, we can use interpolation, which is particularly useful for time series data.
dropped_columns_df['aqi_88502'] = dropped_columns_df['aqi_88502'].interpolate(method='time')

#3. Mean imputation by category:
# If there are still missing values after the above steps, we can use mean imputation based on categories like ‘County’ and ‘date_local’.
# Group by relevant categories and fill with mean
dropped_columns_df['aqi_88502'] = dropped_columns_df.groupby(['county','date_local'])['aqi_88502'].transform(lambda x: x.fillna(x.mean()))

# 4. Overall mean imputation:
# As a last resort, fill any remaining missing values with the overall mean.
dropped_columns_df['aqi_88502'] = dropped_columns_df['aqi_88502'].fillna(dropped_columns_df['aqi_88502'].mean())

# # Check for missing values
print("Missing values after imputation:")
print(dropped_columns_df.isnull().sum())

C:\Users\Chearine\AppData\Local\Temp\ipykernel_25228\2813787990.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dropped_columns_df['aqi_88502'] = dropped_columns_df['aqi_88502'].fillna(method='ffill').fillna(method='bfill')


Missing values after imputation:
latitude     0
longitude    0
county       0
aqi_42101    0
aqi_42401    0
aqi_44201    0
aqi_42602    0
aqi_88502    0
aqi_max      0
dtype: int64


In [25]:
# Write output file to /Resources folder
dropped_columns_df.to_csv("Resources/consolidated_seven_ny_counties_filled.csv", index=False)

